In [42]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import re
import csv
import time
import random
from datetime import date

In [43]:
masterData = pd.DataFrame(columns=['Date','RACE_GROUP','CASE_RATE_ADJ','HOSPITALIZED_RATE_ADJ','DEATH_RATE_ADJ','CASE_COUNT','HOSPITALIZED_COUNT','DEATH_COUNT'])

confirmedProbableData = pd.DataFrame(columns = ['Date','RACE_GROUP','CONFIRMED_DEATH','PROBABLE_DEATH'])

In [44]:
def crawler(url):
    # this function will crawl through all of the next pages of the commit history for the by-race csv, and collect the link for 
    # each page
    html = urlopen(url)
    bs = BeautifulSoup(html, 'html.parser')
    nextPage = bs.find(class_="btn btn-outline BtnGroup-item",text='Older')
    if nextPage.has_attr('href'):
        link = nextPage['href']
        return [url] + crawler(link)
    else:
        return [url]

In [45]:
masterLinks = crawler('https://github.com/nychealth/coronavirus-data/commits/master/totals/by-race.csv')

In [46]:
masterDays = list()
masterCommits = list()

diff = 0
for masterLink in masterLinks:
    # iterate through all of the links found with the crawler function
    global masterDays
    global masterCommits
    html = urlopen(masterLink)
    bs = BeautifulSoup(html, 'html.parser')

    masterDays = masterDays + bs.find_all(class_="f5 text-normal")
    # collect the commit upload dates in a global list called days

    masterCommits = masterCommits + bs.find_all('clipboard-copy', class_="btn btn-outline BtnGroup-item")
    # collects all of the commit addresses/values into a global list called commits by collecting the 
    # commits off of the specific file page, instead of the master page for the entire NYC dataset, we ensure that each 
    # commit actually updated the data we wanted (i.e. doesn't correspond to a different file or README).
    # Additionally, these commit addresses can be used for both the probable-confirmed file, as well as the master by 
    # race files.

    if len(masterDays) != len(masterCommits) and abs(len(masterDays)-len(masterCommits)) != diff:
        diff =  abs(len(masterDays)-len(masterCommits))
        print('DUPLICATE COMMITS FOUND AT: ' + masterLink)
    elif len(masterDays) == len(masterCommits) and diff!=0:
        print('Anomaly at: ' + masterLink)
        # collects the masterLink on which a specific date had multiple uploads, which lead to unequal lengths of each 
        # list
print(len(masterCommits), len(masterDays))

DUPLICATE COMMITS FOUND AT: https://github.com/nychealth/coronavirus-data/commits/master/totals/by-race.csv
8 7


In [47]:
# # clean each list of unnecessary scraped attributes/values
masterDays = [y.text for y in masterDays]

masterCommits = [x['value'] for x in masterCommits]

In [48]:
# above loop always raises duplicate error at 
# https://github.com/nychealth/coronavirus-data/commits/master/totals/by-race.csv, so I manually 
# looked at that page to find which date had multiple commits. It is 11/9, on which for some reason 
# two CSV's were uploaded. I manually found the commits for this date, so I can search through the 
# commit list and find their indexes, and add an additional date to correspond to their indexes in the 
# masterDays list.

i = masterCommits.index('b671e0ed09b458f148110ad0d71479cf62580ea4')
j = masterCommits.index('700a357f272b309fd8841b5e7dc4f4a53bb116ff')

if (i >= len(masterDays)):
    masterDays.append('Commits on Nov 9, 2020')
elif (j >= len(masterDays)):
    masterDays.append('Commits on Nov 9, 2020')
elif (masterDays[i] != 'Commits on Nov 9, 2020'):
    masterDays.insert(i, 'Commits on Nov 9, 2020')
elif (masterDays[j] != 'Commits on Nov 9, 2020'):
    masterDays.insert(j, 'Commits on Nov 9, 2020')

In [49]:
from urllib.error import HTTPError
def commitChecker(commit,url):
    # affirms that each commit actually goes to an existing csv. In the case where a given day had multiple commits 
    # for the same file, only one of those commit addresses would work, and so this function cleans our commits of 
    # any non functional addresses
    url = url.split('[split]')
    try:
        html = urlopen(url[0] + str(commit) + url[1])
    except HTTPError as e:
        index = masterCommits.index(commit)
        print(e)
        print("bad commit: " + commit + ' at ' + str(index))
        print(str(masterDays[index]) + ' ' + commit)
        masterDays.remove(masterDays[index])
        # masterCommits.remove(commit)
        print('removed from lists')
        return False
    else:
        return True

In [50]:
url = 'https://raw.githubusercontent.com/nychealth/coronavirus-data/[split]/totals/by-race.csv'

masterCommits = [c for c in masterCommits if commitChecker(c,url)]

# applies commitChecker function to commits

# mini test to make sure that the lists for commits and days are now of equal length
print("testing: len(commits) == len(days)")
result = len(masterCommits) == len(masterDays)
print("expected result: True")
print("actual result: " + str(result))
print("matches expected? " + str(result == True))
print(len(masterCommits), len(masterDays))

testing: len(commits) == len(days)
expected result: True
actual result: True
matches expected? True
8 8


In [51]:
for day,commit in zip(masterDays,masterCommits):
    # go through the days and commit values and create proper URLs based on the commit and the file name to create 
    # daily dataframes, which we append to a cumulative dataframe, in this instance for the by-race dataset.
    
    df = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/' + commit + '/totals/by-race.csv')
    
    df.insert(0, 'Date', day)
    
    total = {'Date':day, 'RACE_GROUP':'Totals', 'CASE_RATE_ADJ':df['CASE_RATE_ADJ'].sum(),'HOSPITALIZED_RATE_ADJ': df['HOSPITALIZED_RATE_ADJ'].sum(), 'DEATH_RATE_ADJ':df['DEATH_RATE_ADJ'].sum(), 'CASE_COUNT': df['CASE_COUNT'].sum(), 'HOSPITALIZED_COUNT': df['HOSPITALIZED_COUNT'].sum(), 'DEATH_COUNT':df['DEATH_COUNT'].sum()}
    
    # print(total)
    df = df.append(total, ignore_index=True)
    masterData = masterData.append(df, ignore_index=True)

In [52]:
# write the scraped master by-race dataFrame to a csv
with open('./masterDataUpdate.csv', "w", newline='') as f:    
    writer = csv.writer(f, delimiter=',')
masterData.to_csv ('./masterDataUpdate.csv', index=False, header=True)

In [53]:
# confirmedProbableData = pd.DataFrame(columns = ['Date','RACE_GROUP','CONFIRMED_DEATH','PROBABLE_DEATH'])
probUrl = 'https://raw.githubusercontent.com/nychealth/coronavirus-data/[split]/totals/probable-confirmed-by-race.csv'
commitsProb = [c for c in masterCommits if commitChecker(c,probUrl)]

In [54]:
for day,commit in zip(masterDays,commitsProb):
    fd = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/' + commit + '/totals/probable-confirmed-by-race.csv')
    fd.insert(0,'Date',day)
    totals = {'Date':day,'RACE_GROUP':'Totals','CONFIRMED_DEATH':fd['CONFIRMED_DEATH'].sum(),'PROBABLE_DEATH':fd['PROBABLE_DEATH'].sum()}
    fd = fd.append(totals, ignore_index=True)
    confirmedProbableData = confirmedProbableData.append(fd,ignore_index=True)

In [55]:
with open('./confirmedProbableDeathDataUpdate.csv', "w", newline='') as f:    
    writer = csv.writer(f, delimiter=',')
confirmedProbableData.to_csv ('./confirmedProbableDeathDataUpdate.csv', index = False, header=True)

In [56]:
# clean date columns for github csv upload

file_date = masterData['Date'][0].replace(',','').split()
file_date = str(file_date[2]) + '_' + str(file_date[3])

months = dict([('Jan','01'), ('Feb','02'),('Mar','03'),('Apr','04'),('May','05'),('Jun','06'),('Jul','07'),('Aug','08'),('Sep','09'),('Oct','10'),('Nov','11'),('Dec','12')])
# define dates dictionary with the keys being the 3 letter abbreviations used in the commit dates on github, and 
# the values being the appropriate 2 digit numbers for each date

def convert_date(commit):
    vals = commit.split(' ')[2:]
    month = months[vals[0]]
    day = vals[1].replace(',','')
    if len(day) == 1:
        day = '0' + day
    year = vals[2]
    result = year + '-' + month + '-' + day
    return date.fromisoformat(result)

masterData['Date'] = masterData['Date'].apply(convert_date)

confirmedProbableData['Date'] = confirmedProbableData['Date'].apply(convert_date)

with open('./' + file_date + ' - master data by race - NYC.csv', "w", newline='') as out:    
    writer = csv.writer(out, delimiter=',')
masterData.to_csv ('./' + file_date + ' - master data by race - NYC.csv', index = False, header=True)

with open('./' + file_date + ' - confirmed probable deaths by race - NYC.csv', "w", newline='') as outfile:    
    writer = csv.writer(outfile, delimiter=',')
confirmedProbableData.to_csv ('./' + file_date + ' - confirmed probable deaths by race - NYC.csv', index = False, header=True)